In [1]:
!chmod 600 /content/kaggle.json

In [2]:
! KAGGLE_CONFIG_DIR=/content/ kaggle datasets download -d datatattle/covid-19-nlp-text-classification

  0% 0.00/4.38M [00:00<?, ?B/s]
100% 4.38M/4.38M [00:00<00:00, 40.4MB/s]


In [3]:
!mkdir /content/Tweet

In [4]:
import zipfile
zip_file = zipfile.ZipFile('/content/covid-19-nlp-text-classification.zip')
zip_file.extractall('/content/Tweet')

In [31]:
import pandas as pd
import numpy as np

TWEET_DATA = pd.read_csv("/content/Tweet/Corona_NLP_train.csv", nrows=5000)
TWEET_DATA['Tweet'] = TWEET_DATA['OriginalTweet']
TWEET_DATA['Tweet'] = TWEET_DATA['Tweet'].str.lower()
TWEET_DATA.head()

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment,Tweet
0,3799,48751,London,16-03-2020,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/i...,Neutral,@menyrbie @phil_gahan @chrisitv https://t.co/i...
1,3800,48752,UK,16-03-2020,advice Talk to your neighbours family to excha...,Positive,advice talk to your neighbours family to excha...
2,3801,48753,Vagabonds,16-03-2020,Coronavirus Australia: Woolworths to give elde...,Positive,coronavirus australia: woolworths to give elde...
3,3802,48754,NaN,16-03-2020,My food stock is not the only one which is emp...,Positive,my food stock is not the only one which is emp...
4,3803,48755,NaN,16-03-2020,"Me, ready to go at supermarket during the #COV...",Extremely Negative,"me, ready to go at supermarket during the #cov..."


In [32]:
import string 
import re
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist

def remove_special(text):
    text = text.replace('\\t',' ').replace('\\n'," ").replace('\\u'," ").replace('\\'," ")
    text = text.encode('ascii','replace').decode('ascii')
    text = ' '.join(re.sub("([@#]\S+)|(\w+:\/\/\S+)|(\(\W+\))"," ", text).split())
    text = ' '.join(re.sub(r"\d+","",text).split())
    text = ' '.join(re.sub('\s+',' ',text).split())
    text = text.translate(str.maketrans("","",string.punctuation))
    text = text.strip()
    return text.replace("http://"," ").replace("https://", " ")

TWEET_DATA['Tweet'] = TWEET_DATA['Tweet'].apply(remove_special)
TWEET_DATA['Tweet']

0                                                 and and
1       advice talk to your neighbours family to excha...
2       coronavirus australia woolworths to give elder...
3       my food stock is not the only one which is emp...
4       me ready to go at supermarket during the outbr...
                              ...                        
4995    why is government not transmitting benefits of...
4996    as long as were not seeing markets i would cal...
4997    will school fees be refunded if the causes clo...
4998    continues its dominance markets rebounded some...
4999    i guess the new normal is hitting the grocery ...
Name: Tweet, Length: 5000, dtype: object

In [36]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [34]:
def word_tokenize_wrapper(text):
  return word_tokenize(text)

TWEET_DATA['Tweet_Tokens'] = TWEET_DATA['Tweet'].apply(word_tokenize_wrapper)
print('Tokenizing Result : \n')
TWEET_DATA['Tweet_Tokens'].head(5)

Tokenizing Result : 



0                                           [and, and]
1    [advice, talk, to, your, neighbours, family, t...
2    [coronavirus, australia, woolworths, to, give,...
3    [my, food, stock, is, not, the, only, one, whi...
4    [me, ready, to, go, at, supermarket, during, t...
Name: Tweet_Tokens, dtype: object

In [35]:
def freqDist_wrapper(text):
  return FreqDist(text)

TWEET_DATA['review_tokens_fdist'] = TWEET_DATA['Tweet_Tokens'].apply(freqDist_wrapper)
print('Frequency Tokens : \n')
print(TWEET_DATA['review_tokens_fdist'].apply(lambda x : x.most_common()))

Frequency Tokens : 

0                                              [(and, 2)]
1       [(to, 2), (neighbours, 2), (phone, 2), (number...
2       [(coronavirus, 1), (australia, 1), (woolworths...
3       [(food, 2), (is, 2), (not, 2), (you, 2), (stay...
4       [(the, 2), (because, 2), (but, 2), (is, 2), (m...
                              ...                        
4995    [(not, 2), (prices, 2), (why, 1), (is, 1), (go...
4996    [(as, 2), (markets, 2), (open, 2), (long, 1), ...
4997    [(will, 2), (be, 2), (the, 2), (school, 1), (f...
4998    [(continues, 1), (its, 1), (dominance, 1), (ma...
4999    [(the, 2), (by, 2), (i, 1), (guess, 1), (new, ...
Name: review_tokens_fdist, Length: 5000, dtype: object


In [44]:
from nltk.corpus import stopwords
list_stopwords = stopwords.words('english')
list_stopwords.extend(['dont', 'doesnt', 'im', 'youre',
                       'wont','werent','wasnt'])

In [46]:
list_stopwords = set(list_stopwords)

def stopwords_removal(words):
  return [word for word in words if word not in list_stopwords]

TWEET_DATA['Tweet_tokens_WSW'] = TWEET_DATA['Tweet_Tokens'].apply(stopwords_removal)
TWEET_DATA['Tweet_tokens_WSW']

0                                                      []
1       [advice, talk, neighbours, family, exchange, p...
2       [coronavirus, australia, woolworths, give, eld...
3       [food, stock, one, empty, please, panic, enoug...
4       [ready, go, supermarket, outbreak, paranoid, f...
                              ...                        
4995    [government, transmitting, benefits, fall, cru...
4996    [long, seeing, markets, would, call, control, ...
4997    [school, fees, refunded, causes, closures, art...
4998    [continues, dominance, markets, rebounded, ann...
4999    [guess, new, normal, hitting, grocery, store, ...
Name: Tweet_tokens_WSW, Length: 5000, dtype: object

In [47]:
TWEET_DATA['Review_tokens_WSW_fdist'] = TWEET_DATA['Tweet_tokens_WSW'].apply(freqDist_wrapper)
print(TWEET_DATA['Review_tokens_WSW_fdist'].apply(lambda x : x.most_common()))

0                                                      []
1       [(neighbours, 2), (phone, 2), (numbers, 2), (a...
2       [(coronavirus, 1), (australia, 1), (woolworths...
3       [(food, 2), (stay, 2), (stock, 1), (one, 1), (...
4       [(ready, 1), (go, 1), (supermarket, 1), (outbr...
                              ...                        
4995    [(prices, 2), (government, 1), (transmitting, ...
4996    [(markets, 2), (open, 2), (long, 1), (seeing, ...
4997    [(school, 1), (fees, 1), (refunded, 1), (cause...
4998    [(continues, 1), (dominance, 1), (markets, 1),...
4999    [(guess, 1), (new, 1), (normal, 1), (hitting, ...
Name: Review_tokens_WSW_fdist, Length: 5000, dtype: object


In [48]:
TWEET_DATA.Sentiment.value_counts()

Positive              1340
Negative              1263
Neutral                889
Extremely Negative     770
Extremely Positive     738
Name: Sentiment, dtype: int64

In [50]:
TWEET_DATA.shape

(5000, 12)

In [51]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_features=1000)
vectors = vectorizer.fit_transform(TWEET_DATA['Tweet'])

words_df = pd.DataFrame(vectors.toarray(), columns=vectorizer.get_feature_names())
words_df.head()

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,able,about,absolutely,access,according,across,act,action,actually,add,advantage,advice,affected,affecting,afford,after,again,against,ago,agree,air,aisle,aisles,alert,all,allow,allowed,almost,already,also,always,am,amazing,amazon,america,american,americans,amid,amidst,among,...,where,whether,which,while,who,whole,why,wife,will,wipes,wish,with,within,without,woman,wont,word,work,worker,workers,working,works,world,worried,worry,worse,worst,worth,would,wrong,yall,year,years,yes,yesterday,yet,you,your,youre,yourself
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.26833,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.129469,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.149942,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.312457,0.0,0.0,...,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.20372,0.0,0.0,0.0,0.0,0.0,0.13394,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.228506,0.000000,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0


In [52]:
x = words_df
y = TWEET_DATA.Sentiment

In [63]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB

logreg = LogisticRegression(C=1e9, solver='newton-cg', max_iter=1000)
logreg.fit(x, y)
forest = RandomForestClassifier(n_estimators=50)
forest.fit(x,y)
svc = LinearSVC()
svc.fit(x,y)
bayes = MultinomialNB()
bayes.fit(x,y)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/optimize.py:212: ConvergenceWarning: newton-cg failed to converge. Increase the number of iterations.
  ConvergenceWarning,


MultinomialNB()

In [64]:
pd.set_option("display.max_colwidth", 200)

unknown = pd.DataFrame({'content':["I love love love love this kitten",
                                   "I hate hate hate hate this keyboard",
                                   "I'm not sure how I feel about toast",
                                   " i feel that it was sold before"]})
unknown

,content
0,I love love love love this kitten
1,I hate hate hate hate this keyboard
2,I'm not sure how I feel about toast
3,i feel that it was sold before


In [65]:
unknown_vectors = vectorizer.transform(unknown.content)
unknown_words_df = pd.DataFrame(unknown_vectors.toarray(), columns=vectorizer.get_feature_names())
unknown_words_df.head()

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,able,about,absolutely,access,according,across,act,action,actually,add,advantage,advice,affected,affecting,afford,after,again,against,ago,agree,air,aisle,aisles,alert,all,allow,allowed,almost,already,also,always,am,amazing,amazon,america,american,americans,amid,amidst,among,...,where,whether,which,while,who,whole,why,wife,will,wipes,wish,with,within,without,woman,wont,word,work,worker,workers,working,works,world,worried,worry,worse,worst,worth,would,wrong,yall,year,years,yes,yesterday,yet,you,your,youre,yourself
0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.372594,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [66]:
# Prediksi dengan model classifier

# Logistic Regression prediction dan probabilities
unknown['pred_logreg'] = logreg.predict(unknown_words_df)
unknown['pred_logreg_proba'] = logreg.predict_proba(unknown_words_df)[:,1]

# Random Forest prediction dan probabilities
unknown['pred_forest'] = forest.predict(unknown_words_df)
unknown['pred_forest_proba'] = forest.predict_proba(unknown_words_df)[:,1]

# SVC prediction
unknown['pred_svc'] = svc.predict(unknown_words_df)

# Bayes prediction dan probabilities
unknown['pred_bayes'] = bayes.predict(unknown_words_df)
unknown['pred_bayes_probs'] = bayes.predict_proba(unknown_words_df)[:,1]

unknown

,content,pred_logreg,pred_logreg_proba,pred_forest,pred_forest_proba,pred_svc,pred_bayes,pred_bayes_probs
0,I love love love love this kitten,Extremely Positive,1.0,Extremely Positive,0.50,Extremely Positive,Extremely Positive,0.546750
1,I hate hate hate hate this keyboard,Neutral,0.0,Neutral,0.00,Neutral,Positive,0.133537
2,I'm not sure how I feel about toast,Extremely Positive,1.0,Neutral,0.04,Extremely Positive,Positive,0.253916
3,i feel that it was sold before,Extremely Positive,1.0,Neutral,0.12,Neutral,Negative,0.103598
